In [1]:
import math
import os

from pathlib import Path

import numpy as np
import pandas as pd

from star_parser import StarParser, StarTabDf


In [2]:
data_folder = Path(r"/mnt/home/a.murachelli/workspace/StarfileHandler/static/2dshift")
assert data_folder.exists()
star_in = data_folder / "job140_particles.star"
assert star_in.exists()
star_out = data_folder / f"job140_particles_shifted.star"
star_out


PosixPath('/mnt/home/a.murachelli/workspace/StarfileHandler/static/2dshift/job140_particles_shifted.star')

In [3]:
parser = StarParser(star_in)
tabs = parser.parse()
tabs


{'data_optics': StarTable data_optics with 10 columns and 1 record(s),
 'data_particles': StarTable data_particles with 24 columns and 180889 record(s)}

In [4]:
# read pixel size
optics = tabs["data_optics"]
angpix = float(optics.to_df()["ImagePixelSize"])


In [5]:
# choose displacements for each class. Default is 0 (no displacement)
# format: <class_number> : (displX, displY)
"""
Class2: 6.6+ 1+
Class12: 6.6- 2.8+
Class18: 5.6- 1.4-
Class36: 7.8- 3.2+
Class45: 2.2- 5.4+
Class49: 10+ 3+
"""
pixel_displacements_by_class = {
    2: (6.6, 1),
    12: (-6.6, 2.8),
    18: (-5.6, -1.4),
    36: (-7.8, 3.2),
    45: (-2.2, 5.4),
    49: (10, 3),
}
angstrom_displacement_by_class = {
    key: (value[0] * angpix, value[1] * angpix)
    for key, value in pixel_displacements_by_class.items()
}
angstrom_displacement_by_class


{2: (21.779999999999998, 3.3),
 12: (-21.779999999999998, 9.239999999999998),
 18: (-18.479999999999997, -4.619999999999999),
 36: (-25.74, 10.56),
 45: (-7.26, 17.82),
 49: (33.0, 9.899999999999999)}

In [6]:
data = tabs["data_particles"]
##new_origin_X = originX + ((displacementX*(-1)*cos(psi*3.141592/180))-(displacementY*sin((psi*3.141592/180)))  


In [7]:
df = data.to_df().copy()
dispx = 15
dispy = 15
df["OriginXAngst"] = df["OriginXAngst"].astype(float)
df["OriginYAngst"] = df["OriginYAngst"].astype(float)
df["ClassNumber"] = df["ClassNumber"].astype(int)
# Calculate all Psi related data
df["AnglePsiDegrees"] = df["AnglePsi"].astype(float) * math.pi / 180
df["AnglePsiCos"] = df["AnglePsiDegrees"].apply(math.cos)
df["AnglePsiSin"] = df["AnglePsiDegrees"].apply(math.sin)
# set shifts to zero
df["ShiftOriginXAngst"] = 0
df["ShiftOriginYAngst"] = 0
# Calculate shifts only for classes that need it
for selected_class, shifts in angstrom_displacement_by_class.items():
    dispx, dispy = shifts
    print(f"shifting class {selected_class} by (x = {dispx}, y = {dispy}) Angstroms")
    df["ShiftOriginXAngst"] += np.where(
        df["ClassNumber"] == selected_class,
        (-dispx * df["AnglePsiCos"]) + (dispy * df["AnglePsiSin"]),
        0,
    )
    df["ShiftOriginYAngst"] += np.where(
        df["ClassNumber"] == selected_class,
        (dispx * df["AnglePsiSin"]) + (-dispy * df["AnglePsiCos"]),
        0,
    )
# apply shifts
df["NewOriginXAngst"] = df["OriginXAngst"] + df["ShiftOriginXAngst"]
df["NewOriginYAngst"] = df["OriginYAngst"] + df["ShiftOriginYAngst"]


shifting class 2 by (x = 21.779999999999998, y = 3.3) Angstroms
shifting class 12 by (x = -21.779999999999998, y = 9.239999999999998) Angstroms
shifting class 18 by (x = -18.479999999999997, y = -4.619999999999999) Angstroms
shifting class 36 by (x = -25.74, y = 10.56) Angstroms
shifting class 45 by (x = -7.26, y = 17.82) Angstroms
shifting class 49 by (x = 33.0, y = 9.899999999999999) Angstroms


In [8]:
df[df["ClassNumber"] == 2][["OriginXAngst", "NewOriginXAngst", "ClassNumber"]]
# df[df["ClassNumber"] == 2][["OriginXAngst", "NewOriginXAngst", "ClassNumber"]]
# df[df["ClassNumber"] == 3][["OriginXAngst", "NewOriginXAngst", "ClassNumber"]]
# df[df["ClassNumber"] == 4][["OriginXAngst", "NewOriginXAngst", "ClassNumber"]]


,OriginXAngst,NewOriginXAngst,ClassNumber
131651,-2.021880,16.333877,2
131652,4.578123,8.837791,2
131653,1.278123,-15.318777,2
131654,4.578123,19.060069,2
131655,-2.021880,-23.613925,2
...,...,...,...
153792,4.578123,-12.006706,2
153793,-5.321880,12.974910,2
153794,4.578123,-5.228999,2
153795,-5.321880,-17.607221,2


In [9]:
parser.write_out(to_file=True, new_file=star_out)

AttributeError: 'StarParser' object has no attribute 'desination'